##### 저장된 모델 활용
- 모델 파일 종류
    * 가중치 및 절편 저장 파일 => 동일한 구조 모델 인스턴스 생성 후 사용 가능
    * 모델 전체 저장 파일 => 바로 로딩 후 사용 가능


[1] 모듈 로딩 <hr>

In [18]:
# 모듈 로딩

import sys
sys.path.append(r'C:\Users\KDP-14\Desktop\VSCode\KDT6\Torch_NLP\NLP_miniproject\재웅')

import os
import pickle
import pandas as pd
import numpy as np
import torch
from torch import nn


In [19]:
## 모델 파일 관련

# 저장 경로
SAVE_PATH='../NLP_miniproject/model/'

# 모델 구조 및 파라미터 모두 저장 파일명
SAVE_MODEL='model_all.pth'
# SAVE_MODEL='model_score(0.5991)_loss(0.6942).pth'

In [20]:
class SentenceClassifier(nn.Module):
    def __init__(
        self,
        n_vocab,
        hidden_dim,
        embedding_dim,
        n_layers,
        dropout = 0.5,
        bidirectional = True,
        model_type = 'lstm'):

        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )
        if model_type == 'rnn':
            self.model = nn.RNN(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True,
        )
        elif model_type == 'lstm':
            self.model = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True,
            )
        
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2,1)
        else:
            self.classifier = nn.Linear(hidden_dim,1)
        self.dropout = nn.Dropout(dropout)

    def forward(self,inputs):
        embeddings = self.embedding(inputs)
        output,_ = self.model(embeddings)
        last_output = output[:,-1,:]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

[2] 모델 로딩 - 모델 전체 파일 사용

In [21]:
NLP_Model = torch.load(SAVE_PATH+SAVE_MODEL, weights_only=False)

In [22]:
print(NLP_Model)

SentenceClassifier(
  (embedding): Embedding(4052, 128, padding_idx=0)
  (model): LSTM(128, 64, num_layers=5, batch_first=True, dropout=0.5, bidirectional=True)
  (classifier): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


[3] 예측 <hr>

- vocab 로딩하기

In [23]:
# 저장된 vocab 파일 불러오기
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [24]:
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LENGTH=21

In [25]:
def predict_model(model, data, vocab, max_length):
    # 데이터가 문자열이라면 토큰화 및 인덱스로 변환
    tokens = [vocab.get(token, vocab['<unk>']) for token in data]  # 토큰을 인덱스로 변환

    # 길이를 패딩하여 모델의 입력 형태와 일치시키기
    if len(tokens) < max_length:
        tokens = tokens + [vocab['<pad>']] * (max_length - len(tokens))
    else:
        tokens = tokens[:max_length]

    dataTS = torch.LongTensor(tokens).unsqueeze(0)

    # 검증 모드로 모델 설정
    model.eval()
    with torch.no_grad():
        # 추론/평가
        logits = model(dataTS)
        pre_val = torch.sigmoid(logits)

    prediction = (pre_val >= 0.5).float()

    return prediction.item()

In [26]:
type(vocab)

dict

In [33]:
# 예시 : 1
data = "임신 당뇨병 항생제 투여 탄수화물섭취가 많은 경우 경구피임약복용 질병이나 약물투여로 면역기능이 억제될 경우 날씨가 덥거나 환기가 안되는 옷을 입어 질 부위가 열이 나고 축축하고 어두워 진균류 생장의 호조건이 된다"
prediction = predict_model(NLP_Model, data, vocab, MAX_LENGTH)
print("Prediction:", prediction)

Prediction: 1.0


In [34]:
# 예시 : 0
data = "우리나라의  경우  형 간염 바이러스 감염의 장기간 지속이 원인인 경우가 가장 많다"
prediction = predict_model(NLP_Model, data, vocab, MAX_LENGTH)
print("Prediction:", prediction)

Prediction: 0.0


In [35]:
data = "기침하다 저녁 때  시간  정도 후에 열이 오르고 기침을 하며 숨쉬기 어려워한다"
prediction = predict_model(NLP_Model, data, vocab, MAX_LENGTH)
print("Prediction:", prediction)

Prediction: 0.0
